In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
service_endpoint = os.getenv('Endpoint')
api_key = os.getenv('Keys')
deployement_name = os.getenv('DeploymentName')
model_name = os.getenv('ModelName')
api_version = os.getenv('ApiVersion')

In [ ]:
from langchain_openai import AzureChatOpenAI


client = AzureChatOpenAI(
    openai_api_version=api_version,
    temperature=0.0,
    max_tokens=3000,
    azure_endpoint=service_endpoint,
    openai_api_key=api_key,
    deployment_name=deployement_name,
)


In [ ]:
result = client.invoke("what is the capital of uttarakhand?").content
print(result)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

text = "what would be a good company name for a company that makes colorful socks?"

messages = [
    SystemMessage(
        content="You are a helpful assistant that generates company names."
    ),
    HumanMessage(content=text)
]

result = client.invoke(messages).content
print(result)

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that generates company names"),
        ("human", "{text}")
    ]
)

result = chat_prompt.invoke({
    "text": "What would be a good company that makes Electric Cars?"
})

ai_llm_result = client.invoke(result)
print(ai_llm_result.content)

### Straming in Langchain

In [ ]:
# client.stream=True

import sys

for chunk in client.stream("what is the capital of west bengal?"):
    print(chunk.content, end="", flush=True)
    sys.stdout.flush()

## Output Parsing

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke.")
    punchline: str = Field(description="The punchline to the joke.")

class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes.")

parser = PydanticOutputParser(pydantic_object=Joke)




In [ ]:
template = 'Answer the user query.\n{format_instructions}\n{query}'

system_message_prompt = SystemMessagePromptTemplate.from_template(template)

chatprompt = ChatPromptTemplate.from_messages([system_message_prompt])

messages = chatprompt.invoke({
    "query": "What is a really funny joke about LLM?",
    "format_instructions": parser.get_format_instructions()
})

In [ ]:
result = client.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.setup)
    print(joke_object.punchline)
except Exception as e:
    print(e)

In [ ]:
structured_llm = client.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Chat-gpt?")


In [ ]:
print(result.punchline)
print(type(result))